# Loss Given Default Analysis [TPS August]
![](images/unsplash.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@constantinevdokimov?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Konstantin Evdokimov</a>
        on 
        <a href='https://unsplash.com/s/photos/loan?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash.</a> All images are by author unless specified otherwise.
    </strong>
</figcaption>

# 1. Problem definition

In this month's TPS competition, we are tasked to predict the amount of money a bank or a financial institution might lose if a loan goes into default.

Before we start the EDA, let's make sure we are all on the same page on some of the key terms of the problem definition:
1. What is loan default?
   - Default is a failure to repay a debt/loan on time. It can occur when a borrower fails to make timely payments on loans such as mortgage, bank loans, car leases, etc.
2. What is a loss given default (LGD)?
   - LGD is the amount of money a bank or financial institution might lose if a loan goes into default. Calculating and predicting LGD can be complex and involve many factors. 

As you will see in just a bit, the dataset for the competition has over 100 features and the target `loss` is (I think) LGD. For more information on these terms, check out [this](https://www.kaggle.com/c/tabular-playground-series-aug-2021/discussion/256337) discussion thread.

The metric used in this competition is Root Mean Squared Error, a regression metric:
![](images/metric.png)

# 2. Setup

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import rcParams

# Global plot configs
rcParams["figure.dpi"] = 200
rcParams["axes.spines.top"] = False
rcParams["axes.spines.right"] = False

# Pandas global settings
pd.set_option("display.max_columns", None)
pd.set_option("precision", 4)

# Import data
train_df = pd.read_csv("data/train.csv", index_col="id")
test_df = pd.read_csv("data/test.csv", index_col="id")
sub = pd.read_csv("data/sample_submission.csv")

# 3. Overview of the datasets

Both training and test sets have 100 features, excluding the ID column. The target is given as `loss` and has a discrete distribution. 

Some other observations:
- Training and test data contain **250k and 150k** observations, respectively
- There are **no missing values** in both sets
- All features either have `float64` or `int64` type

Here are the first few rows of train and test datasets:

In [18]:
train_df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.0023,59,0.7667,-1.3505,42.2727,16.6857,30.3599,1.2673,0.3920,1.0910,1.9687,1.8746,117.286,6.7162,0.9857,0.9734,4004232,0.3779,1.0338,0.5745,0.1224,8.1862,1517.830,3.1322,0.1128,5.0187,116.766,10891,8.1937,5.7972,1.1000,14.8684,-0.2754,0.9157,167.800,-7.5341,4.2363,1.6296,1.1444,-0.3031,4.0921,3.2262,0.0748,0.2595,289.4920,327.4650,5.3893,7.3948,-0.4899,20.2923,2.4566,1.4477,-10639.0,85.6005,-0.1785,2815,-234.772,1.8332,88.5605,0.3679,8.5753e+09,70.9733,0.3801,0.0318,1.0953,0.5635,0.1227,1.1607,1.6939,1.0722,65.1543,0.0225,-5.6068,1.7987,0.5281,6696.300,-0.5621,1.3010,6.7162,1.1435,2.2998,0.0105,-0.1272,0.2311,4.5161,0.5945,397,0.2640,8.6879,15.0701,0.3766,-42.4399,26.8540,1.4575,0.6962,0.9418,1.8285,0.9241,2.2966,10.4898,15
1,0.7845,145,-0.4638,-0.5304,27324.9000,3.4754,160.4980,0.8280,3.7359,1.2814,-2.7395,-0.5295,157.670,0.6964,1.4412,0.1591,23567462,-0.0896,-0.7116,-1.0459,0.1399,3.6929,-123.354,7.7410,-0.8523,8.5102,161.175,87801,12.0202,1.7839,1.2310,10.1497,-0.0187,1.0113,127.401,11.8214,5.9968,-0.9538,1.3794,1.0795,0.7119,2.9060,0.0512,0.7551,140.8930,29.5252,14.2296,3.5321,-0.4057,42.5357,1.4353,0.9398,138312.0,59.8810,-0.0710,1435,1046.880,1.5677,29.4306,2.4552,4.5182e+09,75.5602,1.9880,0.3182,1.1490,0.7236,0.1848,-0.3483,-7.1763,1.4626,43.1121,-0.0608,64.0455,2.3584,5.7597,3958.140,1.5766,-1.2418,5.9141,0.9598,2.5663,0.0007,-0.3356,-0.2717,5.1032,8.7062,98,0.2105,7.8642,3.3719,0.1480,-184.1320,7.9014,1.7064,-0.4947,-2.0583,0.8192,0.4392,2.3647,1.1438,3
2,0.3178,19,-0.4326,-0.3826,1383.2600,19.7129,31.1026,-0.5154,34.4308,1.2421,2.9018,-0.9603,118.590,7.6964,1.4888,0.3873,235760,0.0556,0.2686,0.7181,0.0271,11.6734,270.247,3.4405,-0.6791,13.3781,150.362,14173,1.6995,7.0473,1.1051,7.6287,0.6857,0.8095,120.064,194.4270,6.7787,0.6341,0.9344,0.9270,0.7411,2.4221,0.2604,0.6266,369.5790,370.0240,4.4889,7.8429,1.1960,43.5343,2.5918,1.2567,168881.0,83.8419,2.7132,2911,23256.900,3.9190,97.5578,4.3854,8.4450e+08,99.4933,5.0980,0.2589,1.1688,0.0491,0.1735,0.9370,2.0595,1.2225,50.2267,-0.0262,71.6849,2.4342,1.9046,27165.800,-0.7732,-1.8334,4.9855,1.1709,1.1720,0.0168,-0.2356,-0.7249,3.2256,4.1710,105,-0.1555,8.9183,0.1863,0.3360,7.4372,37.2181,3.2534,0.3379,0.6150,2.2168,0.7453,1.6968,12.3055,6
3,0.2108,17,-0.6165,0.9464,-119.2530,4.0823,185.2570,1.3833,-47.5214,1.0913,-1.5120,-1.2923,125.461,7.3432,-3.0924,0.7138,1146032,0.3265,0.4548,0.2200,0.0380,5.1520,4893.860,6.8975,-0.8306,4.4318,132.855,77147,32.8090,4.0639,1.1191,3.9178,0.5004,0.2064,120.411,233.5370,7.7513,0.6258,-1.9408,1.3332,-5.7932,1.5651,0.3095,0.1346,84.8637,24.3353,4.8712,2.5510,-0.3723,14.8960,1.1001,0.8922,17006.6,78.4078,0.4257,1723,375.240,1.9454,98.1500,-0.1092,3.5440e+09,60.8082,2.3576,0.2391,1.1612,0.5358,0.2222,1.1631,2.3634,1.4153,116.1820,0.0187,55.4428,2.2282,4.3036,2643.760,-1.6663,0.7924,6.4516,1.0773,2.9068,0.0237,-0.0920,-0.0987,4.2782,5.3475,512,0.8560,8.2766,4.0667,0.3365,9.6678,0.6269,1.4943,0.5175,-10.2221,2.6273,0.6173,1.4565,10.0288,2
4,0.4397,20,0.9681,-0.0925,74.3020,12.3065,72.1860,-0.2340,24.3991,1.1015,1.7735,-0.5468,147.186,17.3943,0.9647,0.9649,19272478,0.1211,0.4225,-0.1031,-0.0009,8.1419,162.713,1.5656,-0.3007,7.5646,160.995,5780,-1.5425,8.0908,1.6058,7.0463,-1.0569,1.7274,126.848,0.9108,6.0887,0.1506,1.1346,1.5535,16.4364,2.4887,0.2292,0.3748,465.2930,76.8593,0.7631,8.2066,-0.6461,72.7674,1.4448,0.8585,-40791.9,70.7998,0.1692,1199,-120.388,1.6879,84.0649,-0.0812,6.3794e+09,103.9900,4.3963,0.2485,1.1808,0.5465,0.1407,0.8044,7.5519,-2.5099,

In [19]:
test_df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
250000,0.8127,15,-1.2391,-0.8933,295.5770,15.8712,23.0436,0.9423,29.8980,1.1139,-1.3122,-1.1100,128.357,7.3445,-1.0306,2.3040,13018510,0.0196,0.7577,-0.7161,0.0223,0.1420,8938.110,6.4437,1.1337,6.5271,148.030,80178,3.4344,5.5352,1.2795,8.2354,-1.3065,1.1107,153.408,50.8583,8.3492,-0.7176,1.9347,0.9745,-1.2989,2.3089,0.0753,0.4911,45.3193,21.0993,11.9258,4.0118,-1.0147,93.4686,2.4723,1.6232,28220.20,79.6277,-0.4354,1651,30.455,1.6514,90.1818,0.5058,2.7264e+09,100.6200,5.1824,0.2645,1.1903,0.5660,0.3546,1.3124,14.3741,0.8395,119.6890,-0.0501,62.5088,1.5597,2.7763,-50.0871,0.2473,0.0353,6.3437,1.1766,2.1044,0.0105,-0.3940,-0.2433,3.7702,4.6354,52,-0.1446,3.8188,-0.3476,0.4464,-422.332,-1.4463,1.6907,1.0593,-3.0106,1.9466,0.5295,1.3869,8.7877
250001,0.1903,131,-0.5014,0.8019,64.8866,3.0970,344.8050,0.8072,38.4219,1.0970,-5.6365,1.2871,128.250,6.6909,1.8363,0.6161,1618530,0.0673,-0.8082,-0.3201,0.9987,15.1139,1432.860,1.5751,-1.0789,2.8047,158.629,78013,0.8695,15.6028,1.2423,5.1889,-1.3847,0.8939,168.000,58.9003,6.4483,1.1952,1.9432,0.9666,-8.5975,2.6715,0.0047,0.4883,151.6100,560.8190,6.2055,5.8434,-0.8574,52.0208,1.6350,0.6180,65940.80,82.5252,0.8063,-17,3257.990,3.5431,64.7339,0.3876,6.1912e+09,112.3230,1.6689,0.2929,1.2335,0.5586,0.2684,0.9802,9.9798,1.8214,-1.3255,0.0475,20.8764,1.8762,3.7224,268.4940,-0.4776,1.6939,6.8224,1.2194,1.2020,-0.0020,-0.3830,0.1241,2.1182,5.1840,70,0.2562,18.0312,23.5723,0.3772,10352.200,21.0627,1.8435,0.2519,4.4406,1.9031,0.2485,0.8639,11.7939
250002,0.9197,19,-0.0574,0.9014,11961.2000,16.3965,273.2400,-0.0033,37.9400,1.1522,-3.4033,0.1426,152.386,15.0852,1.5549,1.6547,17563785,-0.1669,-1.2627,1.4112,0.0040,8.7802,4043.780,7.0511,-1.0129,10.0445,121.983,88912,-0.4870,8.9797,1.1984,8.5821,-0.2936,1.8836,120.411,110.3600,5.1818,1.3658,-1.2579,0.9289,5.2845,2.1028,0.0918,0.5205,82.0779,74.4712,6.0981,2.6587,-0.3393,110.0730,2.4382,0.0134,34353.40,93.1586,3.0899,825,464.775,1.6293,88.2927,0.0995,3.3491e+09,34.8437,5.3338,0.3518,1.3872,0.3924,0.1486,1.0765,5.3435,-2.0625,144.7100,-0.1700,39.3509,2.3231,3.0126,23.2484,0.5114,1.3254,7.3319,1.1737,1.3936,0.0011,-0.1999,0.0274,4.3142,3.2428,501,-0.1414,4.7836,0.0684,0.9901,3224.020,-2.2529,1.5510,-0.5592,17.8386,1.8338,0.9318,2.3369,9.0540
250003,0.8610,19,-0.5495,0.4718,7501.6000,2.8070,71.0817,0.7921,0.3952,1.2016,0.8709,1.2713,157.147,5.8488,1.3856,0.6173,252991,0.1085,0.4793,-0.7198,0.3198,25.2003,-170.328,7.5264,0.7399,0.7369,129.194,15850,11.3217,4.5921,1.3878,4.9169,-0.0106,5.1756,118.858,13.9613,2.3807,0.8835,1.2839,1.4273,3.1218,1.7441,0.2698,0.6378,133.7800,57.7544,3.8178,9.2302,-1.2981,165.7850,1.3772,1.2959,-10886.40,94.3216,-0.2072,160,447.421,1.4996,89.3567,0.8972,2.3641e+09,50.3343,2.2809,0.1202,1.1997,0.6663,0.3741,1.2954,6.9092,-0.8518,0.4072,-0.0574,41.2673,3.0599,2.8497,292.1960,-1.1210,-0.0702,12.1671,1.1670,1.7251,0.0006,-0.6167,-0.3758,6.0540,4.4342,43,-0.2216,8.3863,0.6028,1.3969,9689.760,14.7715,1.4139,0.3293,0.8024,2.2325,0.8933,1.3595,4.8483
250004,0.3132,89,0.5885,0.1677,2931.2600,4.3499,1.5719,1.1183,7.7546,1.1681,0.6714,1.6398,128.023,17.3336,1.9716,0.2521,481327,0.1021,-0.4426,-0.6361,0.0231,13.5070,10276.200,3.3603,-1.4257,5.3158,139.739,10964,3.0741,5.4445,1.1415,7.2739,0.2621,1.2417,118.918,778.0400,7.3407,0.6541,1.2779,0.9838,-4.4679,-0.7915,0.1249,0.1113,174.7860,50.4495,18.3246,7.8915,0.9964,164.2090,1.3630,1.3113,-6608.04,97.9588,-0.1819,709,28748.400,1.6271,102.2770,-0.2482,7.5427e+09,121.8600,4.1176,0.2034,1.2334,0.4639,0.2539,0.6034,